<a href="https://colab.research.google.com/github/ylhz/easy-scraping-tutorial/blob/master/song_craw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 前期环境配置

## 1、安装chromedriver驱动

In [0]:
!apt-get update
!apt install chromium-chromedriver

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromiu

## 2、安装eleenium包



In [0]:
!pip install selenium

## 3、设置环境变量

In [0]:
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [0]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

## 4、装载谷歌云端硬盘

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 代码正式开始

## 1、声明

In [0]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

## 2、加载chrome驱动


> 打开浏览器服务
> 这里隐藏了浏览器，若要显示浏览器爬取过程把该代码块替换成
> ```python
driver = webdriver.Chrome('chromedriver')
```
这样会显示浏览过程

In [0]:

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)


*   计时器



In [0]:
# 计时器
time_start = time.time()

 #模拟浏览器
# options = Options()
# options.add_argument("--headless")
# driver = webdriver.Chrome("../usr/bin/chromedriver",options=options)
#driver = webdriver.Chrome("C:/Program Files (x86)/Google/Chrome/Application/chromedriver.exe")
# driver = webdriver.Chrome('chromedriver')


## 3、全局变量

In [0]:
# 全局变量，控制信息 

# 保存获取的数据
global songlist    
songlist = pd.DataFrame(columns=['songname','singer','华语','欧美','日语','韩语','粤语',
                                '流行','摇滚','民谣','电子','舞曲','说唱','轻音乐','爵士','乡村','R&B/Soul','古典','民族','英伦','金属','朋克','蓝调','雷鬼','世界音乐','拉丁','New Age','古风','后摇','Bossa Nova',
                                '清晨','夜晚','学习','工作','午休','下午茶','地铁','驾车','运动','旅行','散步','酒吧',
                                '怀旧','清新','浪漫','伤感','治愈','放松','孤独','感动','兴奋','快乐','安静','思念',
                                '影视原声','ACG','儿童','校园','游戏','70后','80后','90后','网络歌曲','KTV','经典','翻唱','吉他','钢琴','器乐','榜单','00后'])
songlist = songlist.set_index(['songname','singer'])
 # 迭代控制器
global count   
count = 0
# 递归查询的最大深度
global COUNT_MAX
COUNT_MAX = 4
# 已经导入信息的歌单数目
global list_count    
list_count = 0
# 已经遍历的歌单,存储ID号
global fav    
fav = []
# 用于拼接完整地址
base_url = "https://music.163.com/#"
# 网页起始页
start_url = "https://music.163.com/#/playlist?id=54345191"  #ysm
#start_url = "https://music.163.com/playlist?id=2440796550"
#start_url = "https://music.163.com/#/playlist?id=3206513038"

## 4、自定义函数





### 4.1、获取框架


In [0]:
# 自定义方法
def frame(url):
    """

    进入网址url的框架（id="iframe"），并获取源码后用BesutifulSoup解析

    Args:
        url: str类型的网页完整地址

    Returns:
        soup: BeautifulSoup类型的解析完成后的框架内容
    """
    #print("正在访问："+url)
    driver.get(url)
    driver.switch_to.frame("g_iframe")
    #print (driver.page_source[780:1070])
    time.sleep(1.5)    # 打开后延迟，保证抓取
    html_frame = driver.page_source
    #print("html_frame:\n")
    #print (html_frame[780:1070])
    soup = BeautifulSoup(html_frame,'lxml')
    return soup


### 4.2、获取歌单信息

In [0]:

def song_list(soup):
    """获取歌单信息并保存

    将歌单信息（歌名，歌手，标签）保存到全局变量songlist（type:DataFrame）中，并随即写入文件songlist.xlsx，并返回歌曲链接

    Args:
        soup: (BeautifulSoup)歌单iframe内容

    Returns:
        song_urls: (list)歌单内的所有歌曲完整地址
    """
    song_list = soup.find("table","m-table")

    time.sleep(0.3)
    # 获取歌名
    song_nameb = song_list.find_all("b")
    song_name = []
    for i in song_nameb:
        song_name.append(i.get("title"))    # 获取歌名

    for i in range(len(song_name)):
        song_name[i] = song_name[i].replace("\xa0", ' ')    # 去除特殊空格符


    # 获取歌曲链接
    song_a = song_list.find_all("a",{"href":re.compile("/song\?id=")})
    song_a = str(song_a)
    song_urls = re.findall(r'<a href="(.*?)">',song_a)  # r防止字符转义，保持原有意思
    for i in range(len(song_urls)):    # 拼接完整url
        song_urls[i] = base_url + song_urls[i]
    
    
    # 获取歌手
    song_list = str(song_list)
    singers = re.findall(r'data-res-author="(.*?)"',song_list) # 获取歌手
    

    # 获取歌单标签
    tag_a = soup.find_all("a","u-tag")
    tag_a = str(tag_a)
    song_tag = re.findall(r'&amp;order=hot"><i>(.*?)</i>',tag_a) # 歌单标签


    # 将信息保存到临时变量songlist_tmp 
    songlist_tmp = pd.DataFrame(columns=['songname','singer','华语','欧美','日语','韩语','粤语',
                                 '流行','摇滚','民谣','电子','舞曲','说唱','轻音乐','爵士','乡村','R&B/Soul','古典','民族','英伦','金属','朋克','蓝调','雷鬼','世界音乐','拉丁','New Age','古风','后摇','Bossa Nova',
                                 '清晨','夜晚','学习','工作','午休','下午茶','地铁','驾车','运动','旅行','散步','酒吧',
                                 '怀旧','清新','浪漫','伤感','治愈','放松','孤独','感动','兴奋','快乐','安静','思念',
                                 '影视原声','ACG','儿童','校园','游戏','70后','80后','90后','网络歌曲','KTV','经典','翻唱','吉他','钢琴','器乐','榜单','00后'])
    songlist_tmp.songname = song_name
    songlist_tmp.singer = singers
    for j in song_tag:
        songlist_tmp.loc[:,j] = 1    # 标签计数
    songlist_tmp = songlist_tmp.set_index(['songname','singer'])


    # 存入全局变量并保存到文件
    global songlist
    songlist = songlist.add(songlist_tmp,fill_value=0)
    songlist.to_excel('/content/drive/My Drive/songlist2.xlsx')
    print("    添加完成！！")
# #     songlist = songlist.merge(songlist_tmp)
#     songlist.songname = song_name
#     songlist.singer = singers
#     print(pd.merge(songlist,songlist_tmp,how='left'))

    return song_urls

### 4.3、获取包含该歌的歌单

In [0]:

def favlist(song_url):
    """获取包含该歌曲的歌单地址

    Args: 
        song_url: （str）歌曲的播放页面的地址

    Returns: 
        fav_url: (list)包含传入歌曲的歌单地址列表
    """
    print("favlist")
    soup = frame(song_url)
    fav_ul = soup.find("ul","m-rctlist")
    fav_a = fav_ul.find_all("a",{"class":"sname"})
    fav_url = []
    fav_url_tmp = []
    global fav   # 用于保存已经遍历的歌单
    for i in fav_a:
        fav_href = i.get("href")
        if fav_href not in fav:        # 判断是否已经遍历该歌单
            fav.append(fav_href)    
            fav_url_tmp.append(fav_href)
    for i in range(len(fav_url_tmp)):
        fav_url.append( base_url + fav_url_tmp[i] )
    return fav_url

### 4.4、递归爬取

In [0]:

def getinfo(song_list_url):
    """递归爬取歌单

    筛选合格的歌单，并爬取信息，最后寻找新的歌单

    Args:
        song_list_url: (str)歌单地址
    """
    global count
    print("    当前歌单深度：" + str(count))    # 判断当前迭代次数
    if count < COUNT_MAX:    # 限制递归层数
        songlist_soup = frame(song_list_url)
        #print("二、getinfo成功解析歌单"+str(len(fav))+"："+song_list_url)
#         fav_count = int(favcount(soup))
        fav_count = songlist_soup.find("a","u-btni-fav").get("data-count")    # 获取收藏数目

        print("    fav_count:"+ fav_count)

        if int(fav_count) >= 5000 or count == 0:    # 筛选可信度高的歌单

            #print("三、将当前歌单信息("+song_list_url+")加入标签库")

            song_urls = song_list(songlist_soup)    # 更新歌曲标签库
            global list_count
            list_count +=1

            print("    加入完成，已加入歌单数目：" + str(list_count))

            count +=1
            if count < COUNT_MAX:
                #for i in range(50):           # 遍历歌单的歌曲
                for i in range(len(song_urls)):           # 遍历歌单的歌曲
                    fav_url = favlist(song_urls[i])     # 寻找新的歌单

                    print("当前歌单已经遍历："+str(i)+"   剩余："+str(len(song_urls)-i))

                    for j in fav_url:
                        #count += 1
                        #print(j)
                        
                        getinfo(j)
                        count -= 1

### 4.5、main()

In [0]:

def main():

    print("一、进入start_url(默认最爱歌单)")
    getinfo(start_url)

    #songlist.to_excel('songlist.xlsx')
    # 计时器
    time_end = time.time()
    time_c = time_end - time_start
    print("time cost",time_c,'s')
    print("完成")

## 5、程序入口

In [0]:
if __name__ == '__main__':
    main()

一、进入start_url(默认最爱歌单)
    当前歌单深度：0
    fav_count:0
    添加完成！！
    加入完成，已加入歌单数目：1
favlist
当前歌单已经遍历：0   剩余：690
    当前歌单深度：1
    fav_count:110630
    添加完成！！
    加入完成，已加入歌单数目：2
favlist
当前歌单已经遍历：0   剩余：250
    当前歌单深度：2
    fav_count:1159774
    添加完成！！
    加入完成，已加入歌单数目：3
favlist
当前歌单已经遍历：0   剩余：200
    当前歌单深度：3
    fav_count:569342
    添加完成！！
    加入完成，已加入歌单数目：4
    当前歌单深度：3
    fav_count:514502
    添加完成！！
    加入完成，已加入歌单数目：5
favlist
当前歌单已经遍历：1   剩余：199
    当前歌单深度：3
    fav_count:700205


AttributeError: ignored

## 6、保存相关变量到云盘文件
>将关键变量全部设置为保存到文件，并且也从文件读取初始值，就可以实现续接上次程序执行结果</br>
>每加入一个歌单就记录一次时间，可以看时间递增状况

In [0]:
mydict = {
        'favlist': fav
    }

In [0]:
import json
try:
    with open('/content/drive/My Drive/fav2.json', 'a', encoding='utf-8') as fs:
        json.dump(mydict, fs)
except IOError as e:
    print(e)
print('保存数据完成!')

# 测试中


**大改进**


1.   re和beautifulsoup全部替换为lxml，提升效率
2.   存储思路改变：将DataFrame不合并，首先获取一定数目的歌曲信息，然后用加入的形式（覆盖的反面）写入文件，并在一定周期后合并文件中的信息（将同一首歌的信息合并），这样减小处理dataframe的压力，提高效率
3.   关键变量初始化时用文件初始化，最后也保存到文件




## 尝试多进程分布式爬取

In [0]:
# unseen = set([base_url,])
# seen = set()

# pool = mp.Pool(4)
# count, t1=1, time.time()
# while len(unseen) != 0:
#   # 可以加入限制
#   craw_jobs = [pool.apply_async(craw,args=(url,)) for url in unseen]
#   htmls = [frame(url) for url in unseen]
#   results = [j]